In [14]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [15]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
537,Apartmente'L is one of the most interesting mo...,positive
201,"Blue ribbon banners, stars and stripes forever...",positive
192,An Insomniac's Nightmare is the story of a man...,positive
850,Do you know that they want to escavate the Moo...,negative
903,"I saw ""Sweeney Todd"" on Broadway in 1980. It s...",positive


In [16]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [17]:
import nltk
nltk.download('wordnet')
df = normalize_text(df)
df.head()

[nltk_data] Downloading package wordnet to /home/amar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review,sentiment
537,apartmente l one interesting movie ever seen e...,positive
201,blue ribbon banner star stripe forever decorat...,positive
192,insomniac s nightmare story man s plunge insan...,positive
850,know want escavate moon real and geneve debate...,negative
903,saw sweeney todd broadway starred george hearn...,positive


In [18]:
df['sentiment'].value_counts()

sentiment
negative    260
positive    240
Name: count, dtype: int64

In [19]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [20]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
537,apartmente l one interesting movie ever seen e...,1
201,blue ribbon banner star stripe forever decorat...,1
192,insomniac s nightmare story man s plunge insan...,1
850,know want escavate moon real and geneve debate...,0
903,saw sweeney todd broadway starred george hearn...,1


In [21]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [22]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [25]:
import dagshub
import os

MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
repo_owner = os.getenv("repo_owner")
repo_name = os.getenv("repo_name")

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
dagshub.init(repo_owner=repo_owner, repo_name=repo_name, mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "cse-amarjeet/MLOps-Capstone-Project"

2025-05-08 14:37:00,815 - INFO - Initialized MLflow to track repo "cse-amarjeet/MLOps-Capstone-Project"


Repository cse-amarjeet/MLOps-Capstone-Project initialized!

2025-05-08 14:37:00,818 - INFO - Repository cse-amarjeet/MLOps-Capstone-Project initialized!
2025-05-08 14:37:00,848 - WARNING - Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /cse-amarjeet/MLOps-Capstone-Project.mlflow/api/2.0/mlflow/experiments/get-by-name?experiment_name=Logistic+Regression+Baseline


<Experiment: artifact_location='mlflow-artifacts:/3ab1ce4a965d4c169291415de979bd13', creation_time=1746692111363, experiment_id='1', last_update_time=1746692111363, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [13]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-05-08 13:45:22,372 - INFO - Starting MLflow run...
2025-05-08 13:45:23,028 - INFO - Logging preprocessing parameters...
2025-05-08 13:45:24,671 - INFO - Initializing Logistic Regression model...
2025-05-08 13:45:24,673 - INFO - Fitting the model...
2025-05-08 13:45:24,686 - INFO - Model training complete.
2025-05-08 13:45:24,687 - INFO - Logging model parameters...
2025-05-08 13:45:25,277 - INFO - Making predictions...
2025-05-08 13:45:25,279 - INFO - Calculating evaluation metrics...
2025-05-08 13:45:25,291 - INFO - Logging evaluation metrics...
2025-05-08 13:45:27,333 - INFO - Saving and logging the model...
2025/05/08 13:45:33 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-05-08 13:45:33,063 - INFO - Model training and logging completed in 10.03 seconds.
2025-05-08 13:45:33,064 - INFO - Accuracy: 0.664
2025-05-08 13:45:33,065 - INFO - Precision: 0.6129032258064516
2025-05